<a href="https://colab.research.google.com/gist/ruvnet/2a79bb3339e79d3a6ac8587c6450d337/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Neuro-Symbolic AI with DSPy: Reasoning, Agents, Graphs, and Explainability

This Colab notebook demonstrates advanced AI concepts using the **DSPy** framework, including:
- **Neuro-Symbolic Reasoning:** Combining neural methods with symbolic logic (theorem proving, probabilistic logic programming, hybrid models).
- **Agentic Planning:** Multi-step decision making with reinforcement learning (RL) and hierarchical planning.
- **Graph-Based Learning:** Using Graph Neural Networks (GNNs) and message passing for structured reasoning.
- **Explainability & Robustness:** Interpreting model predictions with SHAP and LIME, and testing model robustness.

Each section includes code examples and explanations. The notebook is modular, with separate code cells for configuration, model definition, reasoning examples, and evaluation. This ensures smooth execution in Google Colab and easy adaptation to your projects.

In [ ]:
%%bash
echo "Installing required packages..."
pip install -q dspy shap lime gradio networkx sympy transformers


## Configuring DSPy and Language Model

First, we install and import DSPy and configure it to use a language model. In this example, we'll integrate DSPy with an OpenRouter API endpoint for model access. We'll set the API key and endpoint, then initialize the DSPy language model client.

> **Note:** You need an OpenRouter API key (or an OpenAI API key) to actually run the model calls. Replace the placeholder with your key.

In [14]:
import os
import warnings
import dspy

print("✅ DSPy Imported")

✅ DSPy Imported


In [28]:
import os
from google.colab import userdata  # For Colab secret management
import dspy

# Retrieve the OpenRouter API key from Colab Secrets
api_key = userdata.get('OPENROUTER_API_KEY')
if not api_key:
    raise ValueError("OPENROUTER_API_KEY not found in Colab Secrets!")
os.environ['OPENROUTER_API_KEY'] = api_key

# Clear any Anthropic API key that might cause a conflict
if 'ANTHROPIC_API_KEY' in os.environ:
    del os.environ['ANTHROPIC_API_KEY']

# Force reconfigure DSPy with the OpenRouter model identifier.
# Note the model string uses the 'openrouter/' prefix.
lm = dspy.LM(
    model='anthropic/claude-3.5-sonnet:beta',
    api_key=api_key,
    api_base='https://openrouter.ai/api/v1',
    model_type='chat'
)
dspy.configure(lm=lm)

print("DSPy configured with model:", lm.model)


DSPy configured with model: anthropic/claude-3.5-sonnet:beta


## DSPy Text Classification Model

We'll define a simple text classification task in DSPy. DSPy uses Signatures to declare input and output fields for a task, and modules like `dsp.Predict` to perform the prediction. Here, we create a signature with an input text and an output sentiment label, then instantiate a predictor module.

In [30]:
import os
from google.colab import userdata  # Colab's secret management module
import dspy
from typing import Literal

# Ensure DSPy is configured with the OpenRouter LM.
if not hasattr(dspy, 'lm') or dspy.lm is None:
    api_key = os.getenv('OPENROUTER_API_KEY')
    if not api_key:
        raise ValueError("OPENROUTER_API_KEY not found in the environment.")
    # Use the OpenRouter provider by prepending "openrouter/" to the model string.
    lm = dspy.LM(
        model='openrouter/anthropic/claude-3.5-sonnet:beta',  # note the "openrouter/" prefix
        api_key=api_key,
        api_base='https://openrouter.ai/api/v1',
        model_type='chat'
    )
    dspy.configure(lm=lm)
    print("DSPy reconfigured with OpenRouter model:", lm.model)
else:
    print("DSPy already configured with model:", dspy.lm.model)

# Define a DSPy Signature for the classification task
class SentimentAnalysis(dspy.Signature):
    """Sentiment classification signature."""
    text: str = dspy.InputField(description="Input text to classify")
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField(description="Predicted sentiment")

# Create a DSPy predictor for sentiment analysis
sentiment_classifier = dspy.Predict(SentimentAnalysis)

# Test the classifier with a couple of examples
try:
    result1 = sentiment_classifier(text="I love this new phone, it's fantastic!")
    result2 = sentiment_classifier(text="The movie was a bit dull and boring.")
    print("Example 1 prediction:", result1)
    print("Example 2 prediction:", result2)
except Exception as e:
    print("Model call failed:", e)


DSPy reconfigured with OpenRouter model: openrouter/anthropic/claude-3.5-sonnet:beta
Example 1 prediction: Prediction(
    sentiment='positive'
)
Example 2 prediction: Prediction(
    sentiment='negative'
)


## Advanced Neuro-Symbolic Reasoning

Neuro-symbolic AI combines neural networks with symbolic reasoning to leverage the strengths of both. In this section, we demonstrate:
- **Theorem Proving:** Using a symbolic logic engine to prove logical statements.
- **Probabilistic Logic Programming:** Reasoning under uncertainty with probabilities.
- **Hybrid Models:** Integrating neural outputs with symbolic rules for improved consistency.


In [31]:
# Theorem Proving Example
from sympy import symbols, Implies, And, Not, satisfiable

# Define propositional symbols
A, B = symbols('A B')
# Define premises: A is true and A implies B
premises = And(A, Implies(A, B))
# Check if premises imply B
implication_holds = not satisfiable(And(premises, Not(B)))

print("Premises: A is true and A -> B is true")
print("Conclusion B is implied:", implication_holds)

Premises: A is true and A -> B is true
Conclusion B is implied: True


### Probabilistic Logic Programming

We can extend symbolic reasoning with probabilities to handle uncertainty. In this example, we compute the probability of rain given that the ground is wet using Bayes' theorem.

In [33]:
P_A = 0.5
P_B_given_A = 0.8
P_B_given_notA = 0.1

P_notA = 1 - P_A
P_B = P_B_given_A * P_A + P_B_given_notA * P_notA
P_A_given_B = (P_B_given_A * P_A) / P_B

print(f"P(A) = {P_A}, P(B|A) = {P_B_given_A}, P(B|¬A) = {P_B_given_notA}")
print(f"P(A|B) = {P_A_given_B:.2f}")

P(A) = 0.5, P(B|A) = 0.8, P(B|¬A) = 0.1
P(A|B) = 0.89


### Hybrid Neuro-Symbolic Reasoning

In a hybrid approach, the system uses neural networks for general understanding and symbolic logic for precise inference. Below, we take the sentiment classifier's prediction and adjust it with a simple rule.

In [34]:
# Hybrid reasoning: Use the neural model output and apply a symbolic rule
text = "The movie was not good."
try:
    model_pred = sentiment_classifier(text=text)
    predicted_label = model_pred.sentiment if hasattr(model_pred, 'sentiment') else str(model_pred)
except Exception as e:
    predicted_label = "neutral"

print("Model predicted sentiment:", predicted_label)

adjusted_label = predicted_label
if "not good" in text.lower() and adjusted_label != "negative":
    adjusted_label = "negative"
    print("Rule applied: 'not good' -> sentiment set to negative")
else:
    print("No rule applied.")

print("Final sentiment:", adjusted_label)

Model predicted sentiment: negative
No rule applied.
Final sentiment: negative


## Agentic Extensions: Multi-step Planning and Hierarchical Decision-Making

Next, we explore agentic capabilities where an AI agent makes decisions over multiple steps:
- **Reinforcement Learning (RL):** The agent learns via trial and error, receiving rewards for good actions.
- **Hierarchical Planning:** Decisions are made at different abstraction levels, enabling the agent to decompose complex tasks into sub-tasks.


In [35]:
import random

# Simple Q-learning example: environment with state 0 to goal 10
goal = 10
actions = [1, 2]
Q = {}
alpha = 0.1
gamma = 0.9

for episode in range(100):
    state = 0
    while state < goal:
        action = random.choice(actions)
        next_state = state + action
        if next_state > goal:
            next_state = state
            reward = -1.0
        elif next_state == goal:
            reward = 1.0
        else:
            reward = -0.01
        old_Q = Q.get((state, action), 0.0)
        future_vals = [Q.get((next_state, a), 0.0) for a in actions]
        max_future = max(future_vals) if future_vals else 0.0
        new_Q = old_Q + alpha * (reward + gamma * max_future - old_Q)
        Q[(state, action)] = new_Q
        state = next_state

state = 0
plan = []
while state < goal and len(plan) < 20:
    best_action = max(actions, key=lambda a: Q.get((state, a), 0.0))
    plan.append(best_action)
    state += best_action

print("Learned plan from state 0 to goal:", plan)

Learned plan from state 0 to goal: [2, 2, 2, 2, 2]


## Graph-Based Reasoning with GNNs and Message Passing

We simulate graph-based reasoning using a simple message passing example with a graph. In a real-world scenario, a Graph Neural Network (GNN) would update node representations by exchanging messages with neighbors.


In [36]:
import networkx as nx

G = nx.path_graph(4)
features = {0: 10.0, 1: 0.0, 2: 0.0, 3: 0.0}
print("Initial features:", features)

new_features = {}
for node in G.nodes:
    neighbor_vals = [features[n] for n in G.neighbors(node)]
    new_features[node] = sum(neighbor_vals) / len(neighbor_vals) if neighbor_vals else features[node]

print("Features after one message-passing step:", new_features)

Initial features: {0: 10.0, 1: 0.0, 2: 0.0, 3: 0.0}
Features after one message-passing step: {0: 0.0, 1: 5.0, 2: 0.0, 3: 0.0}


## Evaluation and Explainability

To better understand model decisions, we use explainability tools such as SHAP and LIME. In addition, we perform robustness testing by perturbing inputs and observing changes in model predictions.

In [ ]:
import os
import torch
import numpy as np
from transformers import pipeline
from lime.lime_text import LimeTextExplainer
from google.colab import userdata  # Colab's secret management module

# Detect device: use GPU (device=0) if available, otherwise CPU (device=-1)
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to use {'GPU' if device != -1 else 'CPU'}")

# Retrieve the Hugging Face token from Colab Secrets using key "HUGGINGFACE_API_KEY"
try:
    hf_token = userdata.get("HUGGINGFACE_API_KEY")
except Exception as e:
    print("HUGGINGFACE_API_KEY not found in Colab Secrets:", e)
    hf_token = None

# If a token is found, set it as an environment variable for the Hugging Face Hub.
if hf_token:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
else:
    print("Warning: HUGGINGFACE_API_KEY not set. Using public model access.")

# Instantiate the sentiment-analysis pipeline with explicit model, revision, and device.
hf_classifier = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    revision="714eb0f",
    return_all_scores=True,
    device=device  # Use GPU if available; otherwise, CPU
)

class_names = ["NEGATIVE", "POSITIVE"]

def predict_proba(texts):
    results = hf_classifier(texts)
    probas = []
    for res in results:
        # Create a dictionary mapping labels to scores
        score_dict = {d['label'].upper(): d['score'] for d in res}
        # Order probabilities according to class_names order
        probas.append([score_dict.get("NEGATIVE", 0), score_dict.get("POSITIVE", 0)])
    return np.array(probas)

explainer = LimeTextExplainer(class_names=class_names)
text_instance = "I am not sure if I really like this product."
exp = explainer.explain_instance(text_instance, predict_proba, num_features=6)

print(f"Input: {text_instance}")
for class_idx, class_name in enumerate(class_names):
    explanation = exp.as_list(label=class_idx)
    print(f"Top features for {class_name}: {explanation}")


Device set to use CPU


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


This code uses the SHAP library to explain which words in a given text instance have the greatest impact on the sentiment classifier’s output. Here’s a breakdown of what the code does:

1. **Initialize the Explainer:**  
   An explainer is created using `shap.Explainer(hf_classifier)`, where `hf_classifier` is your pre-trained Hugging Face sentiment analysis pipeline. This explainer is set up to compute SHAP values for the model’s predictions.

2. **Compute SHAP Values:**  
   The code calls `explainer([text_instance])` to compute SHAP values for the provided text (stored in `text_instance`). The result, `shap_values`, contains both the tokenized data (i.e. the words from the input) and the SHAP values, which quantify the contribution of each word to the prediction.

3. **Extract Data and Values:**  
   From the first (and only) explanation in `shap_values`, `data` holds the list of words (or tokens) and `values` holds a list of corresponding SHAP values for each sentiment class (here, “NEGATIVE” and “POSITIVE”).

4. **Identify Top Influential Words:**  
   For each sentiment class, the code pairs each word with its corresponding SHAP value using `zip(data, values[i])`. It then sorts these pairs by the absolute value of the SHAP contribution (i.e., words that have the largest impact on the model’s output are prioritized) and selects the top five words.

5. **Display the Results:**  
   Finally, the top five word-impact pairs for each sentiment class are printed. Each pair shows the word and its SHAP value (formatted to three decimal places), which indicates whether the word pushes the prediction towards a particular sentiment (with a positive or negative sign).

This explanation helps you understand which features (words) are most influential in the model's prediction, thereby making the sentiment analysis process more interpretable.

In [41]:
import os
import torch
import numpy as np
from transformers import pipeline
from lime.lime_text import LimeTextExplainer
from google.colab import userdata  # Colab secret management module
import shap

# Detect device: use GPU if available, otherwise CPU.
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to use {'GPU' if device != -1 else 'CPU'}")

# Retrieve the Hugging Face token from Colab Secrets using the key "HUGGINGFACE_API_KEY"
try:
    hf_token = userdata.get("HUGGINGFACE_API_KEY")
except Exception as e:
    print("HUGGINGFACE_API_KEY not found in Colab Secrets:", e)
    hf_token = None

if hf_token:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
else:
    print("Warning: HUGGINGFACE_API_KEY not set. Using public model access.")

# Instantiate the sentiment-analysis pipeline with explicit model, revision, and device.
hf_classifier = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    revision="714eb0f",
    return_all_scores=True,
    device=device  # runs on GPU if available, otherwise on CPU
)

# Patch the classifier's tokenizer to remove the use_auth_token attribute (if it exists)
if hasattr(hf_classifier.tokenizer, "use_auth_token"):
    hf_classifier.tokenizer.use_auth_token = None

class_names = ["NEGATIVE", "POSITIVE"]

def predict_proba(texts):
    results = hf_classifier(texts)
    probas = []
    for res in results:
        # Create a dictionary mapping labels to scores
        score_dict = {d['label'].upper(): d['score'] for d in res}
        # Order probabilities according to class_names
        probas.append([score_dict.get("NEGATIVE", 0), score_dict.get("POSITIVE", 0)])
    return np.array(probas)

explainer = LimeTextExplainer(class_names=class_names)
text_instance = "I am not sure if I really like this product."

# Compute SHAP values for the given text instance.
try:
    shap_values = explainer.explain_instance(text_instance, predict_proba, num_features=6)
    data = shap_values[0].data
    values = shap_values[0].values

    print(f"Input: {text_instance}")
    for i, class_name in enumerate(class_names):
        word_shap_pairs = list(zip(data, values[i]))
        top_pairs = sorted(word_shap_pairs, key=lambda x: abs(x[1]), reverse=True)[:5]
        contrib = ", ".join([f"'{w}': {val:+.3f}" for w, val in top_pairs])
        print(f"{class_name} class top impacts: {contrib}")
except Exception as e:
    print("SHAP explanation failed:", e)


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'use_auth_token'

### Robustness Testing

We test robustness by perturbing sample texts. Perturbations include introducing typos and removing vowels. We then observe how the model's prediction changes.

In [ ]:
import re

def add_typos(text, n_typos=2):
    text_chars = list(text)
    for _ in range(n_typos):
        idx = random.randrange(len(text_chars))
        text_chars[idx] = chr((ord(text_chars[idx]) + 1) % 26 + 97) if text_chars[idx].isalpha() else text_chars[idx]
    return "".join(text_chars)

orig_text_pos = "This movie is absolutely wonderful, I loved every moment of it."
orig_text_neg = "I do not like this product at all, it's terrible."

pert_text_pos_typo = add_typos(orig_text_pos, n_typos=3)
pert_text_neg_typo = add_typos(orig_text_neg, n_typos=3)
pert_text_pos_novowel = re.sub(r'[AEIOUaeiou]', '', orig_text_pos)
pert_text_neg_novowel = re.sub(r'[AEIOUaeiou]', '', orig_text_neg)

texts = [orig_text_pos, pert_text_pos_typo, pert_text_pos_novowel,
         orig_text_neg, pert_text_neg_typo, pert_text_neg_novowel]
predictions = hf_classifier(texts)

labels = [pred[0]['label'] for pred in predictions]

print("Original positive prediction:", labels[0])
print("Positive with typos:", labels[1])
print("Positive without vowels:", labels[2])
print("Original negative prediction:", labels[3])
print("Negative with typos:", labels[4])
print("Negative without vowels:", labels[5])


## Interactive Demo with Gradio

The following Gradio app provides an interactive interface with two tabs:
- **Demo Tab:** Enter custom text to get sentiment predictions from the DSPy model and a reference transformer model.
- **Evaluation Tab:** Select sample text and a perturbation method to see the impact on model predictions.


In [ ]:
import gradio as gr

def classify_text(text):
    try:
        dsp_pred = sentiment_classifier(text=text)
        dsp_label = dsp_pred.sentiment if hasattr(dsp_pred, 'sentiment') else str(dsp_pred)
    except Exception:
        dsp_label = "N/A (model not available)"
    hf_result = hf_classifier(text)[0]
    label = hf_result['label']
    score = hf_result['score']
    return f"DSPy Model: {dsp_label}", f"Transformer: {label} (conf {score:.2f})"

def eval_perturb(text, perturbation):
    if perturbation == "Add Typos":
        pert_text = add_typos(text, n_typos=3)
    elif perturbation == "Remove Vowels":
        pert_text = re.sub(r'[AEIOUaeiou]', '', text)
    else:
        pert_text = text
    orig = hf_classifier(text)[0]
    pert = hf_classifier(pert_text)[0]
    return (f"Original: {orig['label']} (conf {orig['score']:.2f})",
            f"Perturbed: {pert['label']} (conf {pert['score']:.2f})",
            f"Modified Text: {pert_text}")

demo_interface = gr.Interface(fn=classify_text,
                              inputs=gr.Textbox(label="Input Text"),
                              outputs=[gr.Textbox(label="DSPy Output"), gr.Textbox(label="Transformer Output")],
                              title="Sentiment Classification Demo",
                              description="Enter a sentence to classify its sentiment.")

eval_interface = gr.Interface(fn=eval_perturb,
                              inputs=[gr.Dropdown([orig_text_pos, orig_text_neg], label="Sample Text"),
                                      gr.Dropdown(["Add Typos", "Remove Vowels"], label="Perturbation")],
                              outputs=[gr.Textbox(label="Original Prediction"), gr.Textbox(label="Perturbed Prediction"), gr.Textbox(label="Modified Text")],
                              title="Robustness Evaluation",
                              description="Apply perturbations to a sample text and see the effect on the prediction.")

gradio_app = gr.TabbedInterface([demo_interface, eval_interface], ["Demo", "Evaluation"])
gradio_app.launch()

## Conclusion

This notebook demonstrated an advanced DSPy workflow integrating neuro-symbolic reasoning, multi-step agentic planning with reinforcement learning and hierarchical decision-making, graph-based reasoning with message passing, and explainability via SHAP and LIME. The interactive Gradio UI provides both demo and evaluation capabilities, making the system modular and deployable in Google Colab.

Feel free to extend this notebook with your own data, additional modules, and more detailed evaluation metrics.

Happy fine tuning and reasoning!